In [1]:
import urllib.request, urllib.parse
from datetime import datetime
import json
import pandas as pd
from datetime import datetime
from datetime import timedelta
import pymysql
from sqlalchemy import create_engine
import logging
import urllib.request, urllib.parse
from datetime import datetime
import json

 
user = 'u177183855_Wk6vI'

passw = 'Freegame@123'

host =  'sql315.main-hosting.eu' 

port = 3306

database = 'u177183855_3em2E'

 
mydb = create_engine('mysql+pymysql://' + user + ':' + passw + '@' + host + ':' + str(port) + '/' + database , echo=False)

 

database = pd.read_sql('SELECT * FROM `Event`',con = mydb)

In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csc_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD


def recommended_list(user_item_matrix, user, N, friends=[], alpha=2):
    """

    :param: user_item_matrix (numpy.ndarray) : The dense user-item matrix.
    :param: user : The user that is being recommended
    :param: friends: A list of friends of the user
    :param: alpha: The weight for changing the friends choice
    :param: N (int) : The number of the most similar users.
    :
    :returns: Top 10 best recommended item (events)
    :rvalue: List of integers
    """

    # if (friends):
    #     print("Friend's experience is being considered")
    # else:
    #     print("Friend's experience is nog being considered")
    (num_users, num_items) = user_item_matrix.shape

    # List of recommended items
    list_of_items = np.zeros(num_items)

    # Get the user-user similarity matrix
    user_user_similarity = cosine_similarity(user_item_matrix)

    # protecting against divide by zero issues
    eps = 1e-6

    for item in range(num_items):

        # Get all the users (indices) that
        # have also reviewed this item.
        # Note: We are removing the current user rating
        others_ratings = np.delete(user_item_matrix, user, 0)
        others_have_rated = np.where(user_item_matrix[:, item] > 0)[0]

        # Get the ratings for the item by the similar users,
        ratings = user_item_matrix[others_have_rated, item]

        # Computing the imilarity between the users and other users.
        sims = user_user_similarity[user, others_have_rated]

        # choose all friends who rated the item
        if friends:
            friends_ratings_matrix = user_item_matrix[friends, :]
            # print("friends_rating_matrix")
            # print(friends_ratings_matrix)
            friends_who_rated_index = np.where(friends_ratings_matrix[:, item] > 0)[0]
            friends_ratings = friends_ratings_matrix[friends_who_rated_index, item]

        # choose top N most similar users
        most_similar_users = sims[np.argsort(sims * -1)][:N]
        most_similar_ratings = ratings[np.argsort(sims * -1)][:N]

        # denominator of user produdct prediction
        norm = np.sum(most_similar_users)
        if norm == 0:
            norm = eps

        # friends ratings

        # prediction of user u on product i: p_{u,i}
        predicted_rating = np.sum(np.multiply(
            most_similar_ratings.T,
            most_similar_users)) / norm

        if (friends):
            if not friends_ratings.size == 0:
                predicted_rating = (predicted_rating + alpha * np.mean(friends_ratings)) / (1 + alpha)

        list_of_items[item] = predicted_rating
    # print(list_of_items)

    return (np.argsort(-list_of_items)[0:9])



# generating social networks of friends
def random_friend_generator(N=500):
    b = np.random.choice([0, 1], (N, N), p=(0.98, 0.02))
    b_symmetric = (b + b.T) / 2
    b_symmetric = b_symmetric + 0.01
    b_symmetric = np.around(b_symmetric)
    b_symmetric = b_symmetric.astype('int')

    graph_as_list = []
    for i in range(N):
        friends = np.where(b_symmetric[i] > 0)[0].tolist()
        graph_as_list.append(friends)

    return graph_as_list



#generating random rating matrix
def random_rating_generator(N=500, M=9):
    user_event_matrix = np.random.choice([0,1,2,3,4,5], (N, M), p=(0.94, 0.01, 0.005, 0.01, 0.015, 0.020))
    ratings_matrix = user_event_matrix
    ratings_matrix = ratings_matrix.astype('float')
    as_list = []
    for i in range(N):
        list_of_rated_resturants  = []
        for j in range(M):
            if ratings_matrix[i,j]:
                list_of_rated_resturants.append((j, ratings_matrix[i,j]))
        as_list.append(list_of_rated_resturants)
    return as_list, ratings_matrix







N = 500

users_recommendation_list = []
[temp , rating_matrix]  =  random_rating_generator(N, 39)
friends_list = random_friend_generator(N)

for i in range(N):
    current_user = recommended_list(rating_matrix, i, 20, friends_list[i], 2)
    users_recommendation_list.append(current_user)


for i in range(N):
  print("\n")
  print("user ", i)
  for event in users_recommendation_list[i]:
      print ("event_", event, end=' ')





user  0
event_ 20 event_ 2 event_ 5 event_ 29 event_ 35 event_ 3 event_ 7 event_ 31 event_ 27 

user  1
event_ 13 event_ 19 event_ 8 event_ 10 event_ 4 event_ 11 event_ 22 event_ 24 event_ 29 

user  2
event_ 26 event_ 24 event_ 14 event_ 2 event_ 35 event_ 32 event_ 37 event_ 10 event_ 8 

user  3
event_ 36 event_ 37 event_ 32 event_ 17 event_ 3 event_ 0 event_ 23 event_ 11 event_ 27 

user  4
event_ 11 event_ 5 event_ 24 event_ 34 event_ 19 event_ 14 event_ 20 event_ 3 event_ 29 

user  5
event_ 17 event_ 29 event_ 21 event_ 19 event_ 33 event_ 18 event_ 36 event_ 8 event_ 15 

user  6
event_ 25 event_ 21 event_ 20 event_ 8 event_ 22 event_ 3 event_ 31 event_ 17 event_ 34 

user  7
event_ 29 event_ 12 event_ 22 event_ 15 event_ 14 event_ 17 event_ 25 event_ 34 event_ 36 

user  8
event_ 21 event_ 8 event_ 36 event_ 10 event_ 14 event_ 11 event_ 34 event_ 16 event_ 5 

user  9
event_ 8 event_ 3 event_ 2 event_ 22 event_ 17 event_ 11 event_ 19 event_ 23 event_ 21 

user  10
event_ 35

event_ 17 event_ 37 event_ 11 event_ 2 event_ 12 event_ 30 event_ 14 event_ 35 event_ 18 

user  90
event_ 5 event_ 17 event_ 31 event_ 22 event_ 14 event_ 8 event_ 34 event_ 4 event_ 12 

user  91
event_ 17 event_ 11 event_ 3 event_ 19 event_ 5 event_ 37 event_ 13 event_ 24 event_ 0 

user  92
event_ 19 event_ 28 event_ 4 event_ 7 event_ 20 event_ 15 event_ 14 event_ 27 event_ 5 

user  93
event_ 0 event_ 17 event_ 34 event_ 35 event_ 13 event_ 38 event_ 21 event_ 2 event_ 31 

user  94
event_ 34 event_ 10 event_ 22 event_ 11 event_ 31 event_ 21 event_ 20 event_ 3 event_ 24 

user  95
event_ 22 event_ 31 event_ 10 event_ 32 event_ 8 event_ 16 event_ 23 event_ 15 event_ 17 

user  96
event_ 11 event_ 37 event_ 4 event_ 14 event_ 31 event_ 35 event_ 38 event_ 10 event_ 17 

user  97
event_ 17 event_ 13 event_ 15 event_ 4 event_ 25 event_ 28 event_ 6 event_ 22 event_ 34 

user  98
event_ 1 event_ 36 event_ 4 event_ 5 event_ 8 event_ 20 event_ 15 event_ 2 event_ 0 

user  99
event_ 28 eve

event_ 37 event_ 17 event_ 3 event_ 24 event_ 34 event_ 38 event_ 36 event_ 19 event_ 16 

user  174
event_ 19 event_ 21 event_ 17 event_ 8 event_ 4 event_ 30 event_ 3 event_ 34 event_ 0 

user  175
event_ 35 event_ 31 event_ 21 event_ 10 event_ 11 event_ 6 event_ 16 event_ 20 event_ 30 

user  176
event_ 10 event_ 29 event_ 9 event_ 35 event_ 37 event_ 20 event_ 36 event_ 28 event_ 33 

user  177
event_ 12 event_ 3 event_ 14 event_ 10 event_ 5 event_ 8 event_ 22 event_ 20 event_ 31 

user  178
event_ 11 event_ 28 event_ 5 event_ 18 event_ 20 event_ 21 event_ 30 event_ 17 event_ 6 

user  179
event_ 0 event_ 27 event_ 34 event_ 33 event_ 9 event_ 8 event_ 24 event_ 13 event_ 20 

user  180
event_ 17 event_ 18 event_ 24 event_ 20 event_ 5 event_ 36 event_ 15 event_ 37 event_ 3 

user  181
event_ 16 event_ 15 event_ 14 event_ 35 event_ 32 event_ 26 event_ 12 event_ 19 event_ 17 

user  182
event_ 35 event_ 8 event_ 29 event_ 36 event_ 23 event_ 3 event_ 30 event_ 6 event_ 24 

user  183


user  257
event_ 37 event_ 12 event_ 28 event_ 2 event_ 24 event_ 27 event_ 6 event_ 8 event_ 16 

user  258
event_ 28 event_ 13 event_ 22 event_ 27 event_ 2 event_ 32 event_ 14 event_ 10 event_ 24 

user  259
event_ 20 event_ 24 event_ 27 event_ 12 event_ 33 event_ 26 event_ 37 event_ 6 event_ 25 

user  260
event_ 11 event_ 37 event_ 3 event_ 35 event_ 32 event_ 0 event_ 10 event_ 14 event_ 5 

user  261
event_ 0 event_ 31 event_ 29 event_ 23 event_ 18 event_ 16 event_ 15 event_ 19 event_ 1 

user  262
event_ 32 event_ 26 event_ 15 event_ 12 event_ 34 event_ 21 event_ 0 event_ 8 event_ 30 

user  263
event_ 15 event_ 4 event_ 9 event_ 10 event_ 35 event_ 28 event_ 21 event_ 11 event_ 14 

user  264
event_ 28 event_ 20 event_ 19 event_ 24 event_ 22 event_ 16 event_ 12 event_ 15 event_ 13 

user  265
event_ 29 event_ 4 event_ 33 event_ 35 event_ 22 event_ 15 event_ 25 event_ 23 event_ 8 

user  266
event_ 22 event_ 4 event_ 31 event_ 30 event_ 17 event_ 21 event_ 0 event_ 19 event_ 18 

event_ 37 event_ 19 event_ 17 event_ 13 event_ 22 event_ 10 event_ 14 event_ 0 event_ 2 

user  409
event_ 17 event_ 0 event_ 34 event_ 31 event_ 22 event_ 38 event_ 8 event_ 11 event_ 2 

user  410
event_ 35 event_ 24 event_ 31 event_ 13 event_ 18 event_ 16 event_ 4 event_ 22 event_ 27 

user  411
event_ 24 event_ 8 event_ 4 event_ 14 event_ 9 event_ 17 event_ 29 event_ 36 event_ 20 

user  412
event_ 19 event_ 36 event_ 29 event_ 3 event_ 28 event_ 7 event_ 11 event_ 14 event_ 13 

user  413
event_ 17 event_ 14 event_ 19 event_ 9 event_ 4 event_ 5 event_ 16 event_ 36 event_ 11 

user  414
event_ 37 event_ 14 event_ 10 event_ 11 event_ 15 event_ 4 event_ 20 event_ 36 event_ 22 

user  415
event_ 11 event_ 21 event_ 17 event_ 8 event_ 16 event_ 14 event_ 15 event_ 22 event_ 1 

user  416
event_ 1 event_ 23 event_ 35 event_ 17 event_ 14 event_ 13 event_ 24 event_ 11 event_ 30 

user  417
event_ 24 event_ 4 event_ 17 event_ 19 event_ 31 event_ 11 event_ 26 event_ 32 event_ 37 

user  418